In [8]:
import pandas.io.gbq
with open("bigquery_project.txt") as f:
    projectid = f.read().strip()
pandas.__version__

'0.18.0'

In [21]:
data_range = 'TABLE_DATE_RANGE([the-psf:pypi.downloads], TIMESTAMP("20160530"), TIMESTAMP("20160605"))'
count_query = """
SELECT count(*), count(distinct file.filename), count(distinct file.project)
FROM {}
""".format(data_range)
total_downloads, distinct_files, distinct_projects = pandas.io.gbq.read_gbq(count_query, projectid).iloc[0]
print("Total downloads in time period: {:,d}".format(total_downloads))
print("Unique files downloaded in time period: {:,d}".format(distinct_files))
print("Unique projects with files downloaded in time period: {:,d}".format(distinct_projects))

Requesting query... ok.
Query running...
Query done.
Cache hit.

Retrieving results...
Got 1 rows.

Total time taken 1.44 s.
Finished at 2016-06-09 16:02:47.
Total downloads in time period: 101,868,831
Unique files downloaded in time period: 627,213
Unique projects with files downloaded in time period: 69,910


In [2]:
data_query = """
SELECT *
FROM {}
WHERE file.project = "contextlib2"
""".format(data_range)
df = pandas.io.gbq.read_gbq(data_query, projectid)
df.to_csv("contextlib2_downloads_20160522_20160529.csv")

Requesting query... ok.
Query running...
  Elapsed 12.35 s. Waiting...
Query done.
Processed: 25.1 Gb

Retrieving results...
  Got page: 1; 20% done. Elapsed 17.05 s.
  Got page: 2; 40% done. Elapsed 22.64 s.
  Got page: 3; 59% done. Elapsed 27.44 s.
  Got page: 4; 79% done. Elapsed 32.44 s.
  Got page: 5; 99% done. Elapsed 37.3 s.
  Got page: 6; 100% done. Elapsed 38.49 s.
Got 84333 rows.

Total time taken 42.03 s.
Finished at 2016-06-09 15:35:30.


In [6]:
df = pandas.read_csv("contextlib2_downloads_20160522_20160529.csv")
df.head()

,Unnamed: 0,timestamp,country_code,url,file_filename,file_project,file_version,file_type,details_installer_name,details_installer_version,...,details_implementation_version,details_distro_name,details_distro_version,details_distro_id,details_distro_libc_lib,details_distro_libc_version,details_system_name,details_system_release,details_cpu,details_openssl_version
0,0,2016-05-25 12:17:54,PL,/packages/e6/02/0db6f0ed807f7fa9c005cc0e2ebb3d...,contextlib2-0.5.3-py2.py3-none-any.whl,contextlib2,0.5.3,bdist_wheel,pip,8.1.2,...,2.7.9,debian,8.4,NaN,glibc,2.4,Linux,3.16.0-4-amd64,x86_64,OpenSSL 1.0.1k 8 Jan 2015
1,1,2016-05-25 12:19:30,US,/packages/1e/82/8fa4e44f849237b13b6631b6a97569...,contextlib2-0.5.1.tar.gz,contextlib2,0.5.1,sdist,pip,8.1.2,...,2.7.5,CentOS Linux,7.2.1511,Core,glibc,2.2.5,Linux,3.10.0-327.13.1.el7.x86_64,x86_64,OpenSSL 1.0.1e-fips 11 Feb 2013
2,2,2016-05-25 12:56:21,RU,/packages/e6/02/0db6f0ed807f7fa9c005cc0e2ebb3d...,contextlib2-0.5.3-py2.py3-none-any.whl,contextlib2,0.5.3,bdist_wheel,pip,8.1.2,...,2.7.11,Ubuntu,14.04,trusty,glibc,2.4,Linux,3.13.0-43-generic,x86_64,OpenSSL 1.0.1f 6 Jan 2014
3,3,2016-05-25 12:31:18,US,/packages/e6/02/0db6f0ed807f7fa9c005cc0e2ebb3d...,contextlib2-0.5.3-py2.py3-none-any.whl,contextlib2,0.5.3,bdist_wheel,pip,7.1.2,...,2.7.10,debian,jessie/sid,NaN,glibc,2.7,Linux,3.19.0-30-generic,x86_64,NaN
4,4,2016-05-25 12:55:04,US,/packages/e6/02/0db6f0ed807f7fa9c005cc0e2ebb3d...,contextlib2-0.5.3-py2.py3-none-any.whl,contextlib2,0.5.3,bdist_wheel,pip,1.5.4,...,2.7.6,NaN,NaN,NaN,NaN,NaN,Linux,3.13.0-85-generic,NaN,NaN


In [4]:
100 * df["details_python"].value_counts(normalize=True)[:20]

2.7.6      40.964984
2.7.9      10.277116
2.7.11      8.603987
2.7.3       6.449433
2.7.10      6.209906
3.4.3       5.780655
2.7.5       4.751402
3.5.1       4.292507
2.7         2.672738
2.7.11+     1.894869
3.4.2       1.667200
2.6.6       1.343484
3.5.0       1.145459
3.4.4       0.794470
3.5.1+      0.544271
3.4.1       0.420950
2.7.8       0.305930
3.3.5       0.234784
2.7.2       0.192096
3.4.0       0.115020
Name: details_python, dtype: float64

In [5]:
100 * df.where(df["details_python"] == "2.7.6")["details_installer_version"].value_counts(normalize=True, dropna=False)[:20]

NaN      59.035016
8.1.2    24.608398
1.5.4     6.187376
7.1.2     4.261677
1.4.1     3.761280
8.1.1     0.873916
6.0.8     0.362847
8.0.2     0.273914
1.5.6     0.227669
6.1.1     0.104348
7.1.0     0.094862
8.1.0     0.067589
7.0.3     0.032016
8.0.0     0.027273
8.0.3     0.026087
6.0.6     0.016601
7.0.0     0.010672
1.5.5     0.005929
7.0.1     0.005929
6.0       0.005929
Name: details_installer_version, dtype: float64